In [19]:

import json
import pprint
import time
import threading
import re
import ctypes 

from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
html = None

#hardcoded the urls
urls = {
    "https://www.tesla.com/inventory/used/ms?arrangeby=relevance&zip=*",
    "https://www.tesla.com/inventory/used/m3?arrangeby=relevance&zip=*",
    "https://www.tesla.com/inventory/used/mx?arrangeby=relevance&zip=*",
    "https://www.tesla.com/inventory/used/my?arrangeby=relevance&zip=*"
    }

#below container contains the data we want to scrape,its usually the top most tag.

results_container_selector = 'div.results-container.results-container--grid.results-container--has-results'
delay = 400  # seconds

#below are the features im going to scrape, using dictionary and key
car={'price':[], 'year':[],'car_type':[],'trim':[],'mileage':[],'location':[],
     'transport_fee':[],'emi':[],'DownPay':[],'Emi_Span':[],'APR':[],
     'EPA_Range':[],'Top_Speed':[],'60mph_time':[],   
      'colour':[],'wheels':[],'interior':[],'Highlights':[]
    
    }

for city in urls:

    try:

        print(datetime.now().strftime("%H:%M:%S") + " Searching Tesla's website in " + city)
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        browser = webdriver.Chrome('C:/Users/STSC/OneDrive - horizon.csueastbay.edu/Documents/Semester II/Data Analytics/Project 2023/chromedriver.exe', options=chrome_options)
        browser.get(city)

        # wait for results to be displayed
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((By.CSS_SELECTOR,results_container_selector))
        )

    except TimeoutException:
        print('Loading took too much time!')
    else:
        html = browser.page_source
    finally:
        browser.quit()

    if html:
        soup = BeautifulSoup(html, 'lxml')
        for div in soup.find_all("div", {'class':'alertbox full-width'}): 
            div.decompose()
        #results_container_selector
        #for car_html in soup.select_one('results-container.results-container--grid.results-container--has-results').findChildren('article'):
        for car_html in soup.find_all('article'):
            try:

                price = int(re.sub('[^0-9]','', car_html.select_one('section.result-header').select_one('div.result-pricing').select_one('div.result-price').select_one('span.result-purchase-price.tds-text--h4').text.replace('$', '').replace(',', '')))

            except:

                price = "0"

            try:
                colour = car_html.select('section.result-features.features-grid')[0].select('ul')[1].select('li')[0].text.replace('Paint','')                                                
            except:
                colour ="N/A"
            try:
                year = car_html.select_one('section.result-header').select_one('div.result-basic-info').select_one('h3.tds-text--h4').select('span')[0].text
                car_type = car_html.select_one('section.result-header').select_one('div.result-basic-info').select_one('h3.tds-text--h4').select('span')[1].text
            except:  
                car_type = "N/A"
            try:    
                trim = car_html.select_one('section.result-header').select_one('div.result-basic-info').select('div')[0].text
            except:
                trim = "N/A"
            try:
                mileage = int(car_html.select_one('section.result-header').select_one('div.result-basic-info').select('div')[1].text.replace('Less than ', '').replace(' mile odometer', '').replace(',', ''))
            except:
                mileage = "0"
            try:
                location = car_html.select_one('section.result-header').select_one('div.result-basic-info').select('div')[2].text
            except:
                location = "N/A"
            try:
                wheels = re.sub('[^0-9]', '', car_html.select('section.result-features.features-grid')[0].select('ul')[1].select('li')[1].text) + " inch wheels"
            except:
                wheels = "N/A"
            try:
                interior = car_html.select('section.result-features.features-grid')[0].select('ul')[1].select('li')[2].text
            except:
                interior = "N/A"
            try:
                emi     =   car_html.select_one('section.result-header').select_one('div.result-pricing').select_one('div.tds-text--headline_color.tds-text--caption"').select('div').text#result-loan-payment').text.replace('$', '')
            except:
                emi  = "N/A"
            try:
                t_fee   =  car_html.select_one('section.result-header').select_one('div.result-pricing').select_one('div.tds-text--caption')[0].text#.replace('No Est. Transport Fee:','0').replace('$', '')))
            
            except:
                t_fee  = "N/A"

            try:
                downPay =  int(re.sub('[^0-9]','', car_html.select_one('section.result-header').select_one('div.result-pricing').select_one('div.tds-text--headline_color.tds-text--caption').select_one('div.financing-tooltip').select('span')[0].text))
            except:
                downPay  = "N/A"
            try:
                PaySpan =  int(re.sub('[^0-9]','', car_html.select_one('section.result-header').select_one('div.result-pricing').select_one('div.tds-text--headline_color.tds-text--caption').select_one('div.financing-tooltip').select('span')[1].text))
            except:
                PaySpan  = "N/A"
            try:
                Apr     =  int(re.sub('[^0-9]','', car_html.select_one('section.result-header').select_one('div.result-pricing').select_one('div.tds-text--headline_color.tds-text--caption').select_one('div.financing-tooltip').select('span')[2].text))
            except:
                Apr  = "N/A"
            try:
                epa_range = int(car_html.select_one('section.result-highlights-cta').select_one('div.result-highlights').select_one('ul.highlights-list').select('span.tds-text--h4')[0].text)
            except:
                epa_range = 0
            try:    
                top_speed = int(car_html.select_one('section.result-highlights-cta').select_one('div.result-highlights').select_one('ul.highlights-list').select('span.tds-text--h4')[1].text)
            except:
                top_speed = 0
            try:    
                timeto    = float(car_html.select_one('section.result-highlights-cta').select_one('div.result-highlights').select_one('ul.highlights-list').select('span.tds-text--h4')[2].text)
            except:
                timeto  = 0
     #   Append collected data into a list
            car['price'].append(price)
            car['colour'].append(colour)
            car['car_type'].append(car_type)
            car['year'].append(year)
            car['trim'].append(trim)
            car['mileage'].append(mileage)
            car['location'].append(location)
            car['wheels'].append(wheels)
            car['interior'].append(interior)
           
            car['transport_fee'].append(t_fee)
            car['emi'].append(emi)
            car['DownPay'].append(downPay)
            car['Emi_Span'].append(PaySpan)
            car['APR'].append(Apr)
            car['EPA_Range'].append(epa_range)
            car['Top_Speed'].append(top_speed)
            car['60mph_time'].append(timeto) 
        
            time.sleep(20) # seconds
print("Data Scrapped")




22:14:20 Searching Tesla's website in https://www.tesla.com/inventory/used/mx?arrangeby=relevance&zip=*


C:\Users\STSC\AppData\Local\Temp\ipykernel_114896\3905104442.py:46: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/STSC/OneDrive - horizon.csueastbay.edu/Documents/Semester II/Data Analytics/Project 2023/chromedriver.exe', options=chrome_options)


22:15:32 Searching Tesla's website in https://www.tesla.com/inventory/used/m3?arrangeby=relevance&zip=*
22:32:23 Searching Tesla's website in https://www.tesla.com/inventory/used/my?arrangeby=relevance&zip=*
22:34:56 Searching Tesla's website in https://www.tesla.com/inventory/used/ms?arrangeby=relevance&zip=*
Data Scrapped


In [18]:
import pandas as pd
df=pd.DataFrame.from_dict(car, orient='index') #convert dictionary to dataframe and add Nan to unfilled values
df=df.T #transpose dataframe
#Data Cleaning
#drop all null and rest index
#df = df.dropna()
df.reset_index(inplace=True, drop=True)
df.to_csv('Tesla_Inventory_new.csv') #Convert dataframe to csv file

In [ ]:
car['60mph_time']